In [ ]:
using Plots
using FFTW

In [ ]:
# reverse a signal about the first element
flip(x) = x[-(0:length(x)-1) .& (length(x)-1) .+ 1]
swap(x) = conj(x)*im

linphase(offs, n) = cispi.(offs*-2(0:n-1)/n)
shift(x, offs) = x.*linphase(offs, length(x))

In [ ]:
N = 8
x, y = rand(0:9, N) + rand(0:9, N)im, rand(0:9, N) + rand(0:9, N)im
X, Y = fft(x), fft(y)
0

# FFT Identities

$x$ is a signal with elements from $\{x_n\}, n \in (1, N]$

$F(x) = X$

## Linearity

Addition and scalar multiplication work as expected.

$F(a \cdot x + b \cdot y) = a \cdot F(x) + b \cdot F(y)$

In [ ]:
round.([fft(2x + 3y) 2*fft(x) + 3*fft(y)]; digits=2)

## Flipping Time and Frequency

Flipping a signal around the first element will be denoted as $x^!$. Ex:

$x = {a, b, c, d}$

$x^! = {a, d, c, b}$

Flipping time flips frequency.

$F(x)^! = F(x^!)$

In [ ]:
round.([(flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Flipping via Conjugation

Conjugating the signal and spectrum is equal to flipping.

$F(x^*)^* = F(x^!) = F(x)^!$

In [ ]:
round.([(conj∘fft∘conj)(x) (flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Flipping via Swapping

Swapping the real/imaginary parts of the signal and spectrum is equal to flipping. This is useful if you have separate arrays for real/imag.

$F(x^* \cdot i)^* \cdot i = F(x^!) = F(x)^!$

In [ ]:
round.([(swap∘fft∘swap)(x) (flip∘fft)(x) (fft∘flip)(x)]; digits=2)

## Conjugate in Time

Conjugate in time is equal to the conjugate of the reverse in freq.

$F(x^*) = F(x^!)^* = F(x)^{!*} = F(x)^{*!}$

In [ ]:
round.([(fft∘conj)(x) (conj∘fft∘flip)(x) (conj∘flip∘fft)(x) (flip∘conj∘fft)(x)]; digits=2)

## Conjugate in Freq

Conjugate in freq is equal to the conjugate of the reverse in time.

$F(x)^* = F(x^*)^! = F(x^{*!}) = F(x^{!*})$

<!-- Swapping re/im acts similar to conjugation. (Note: I feel like there was a better identity to use here...)

`conj(fft(x)*im) = flip(conj(fft((x)))*im)` -->

In [ ]:
round.([(conj∘fft)(x) (flip∘fft∘conj)(x) (fft∘flip∘conj)(x) (fft∘conj∘flip)(x)]; digits=2)

## Almost it's own inverse

Same as FFT, but flipped and scaled.

$F^{-1}(x) \cdot N = F(x^!) = F(x)^! = F(x^*)^* = F(x^* \cdot i)^* \cdot i$

Always scaling the result by $\sqrt{N}$ is useful to avoid the iFFT scaling issue.

In [ ]:
round.([(ifft)(x)*N (fft∘flip)(x) (flip∘fft)(x) (conj∘fft∘conj)(x) (swap∘fft∘swap)(x)]; digits=2)

## Real/Imaginary Valued FFTs

Real valued FFTs have a symmetric result. ex:

$F(x) = [a, b, c, d, c^*, b^*, a^*]$

## Zero padding

Padding out a signal with zeros doesn't add any addtional information. It will increase the frequency resolution of the fft without adding any addition frequencies to it.

## Repetition

If you have a signal $x$, it will have an FFT of the form:

$F(x) = \{a, b, c, d\}$

If you repeat the signal, the even values will all be zeros. This is because the two halves are the same except for the phase. The even values are rotated by 180° so they cancel out.

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, d, 0\}$

If $x$ is real, then you get the usual symmetry as well:

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, b^*, 0\}$

## Other Sequences

Reversing a signal will be denoted as $x^@$. Ex:

$x = {a, b, c, d}$

$x^@ = {d, c, b, a}$

$F(\{x, x^@\}) = \{e, f, g, h, 0, -h, -g, -f\}$

In [ ]:
f() = fft(reverse(x))
g() = reverse(fft(x))
round.(f() - g(); digits = 10)

plot(
    plot([real(f()), real(g())]),
    plot([imag(f()), imag(g())]),
    plot([abs.(f()), abs.(g())]),
    label = ["f" "g"]
)

In [ ]:
plot()

xx = [x; reverse(x)]

fft((xx)).*linphase(0.5, 2N)

In [ ]:
z = zeros(ComplexF64, 2N)
z[2:2:2N] .= real(x)
Z = fft(z)

plot(
    plot(real(Z)),
    plot(imag(Z)),
    plot(abs.(Z)),
    label = ["f" "g"]
)
round.(Z; digits=3)

Z[0N + 1:1N] + Z[1N + 1:2N]
# [x; -x] sequence?

In [ ]:
N = 8
# x = Array(1:N)
# x = Array(1:N).^2
x = rand(-9:9, N) + rand(-9:9, N)*im
# x = cos.((0:N-1).*(6π/N))
# plot(x)

y = fft(x)
# plot([x, abs.(y)])
round.([x y]; digits=2)


# Calculating a complex valued FFT

In [ ]:
# calculate FFT of odd elements
xo = Array(x)
xo[2:2:N] .= 0
yo = fft(xo)

# calculate FFT of even elements
xe = Array(x)
xe[1:2:N] .= 0
ye = fft(xe)

# FFT is linear, you can add the results
round.([yo ye yo + ye]; digits=2)

The FFT of a sequence with even elements 0s is the FFT of the odd elements repeated twice.

The FFT of a sequence with odd elements 0s is the FFT of the evens phase shifted by half a sample and repeated twice with the second repetition negated.

In [ ]:
# fft of evens and odds without zeros inserted
yo = fft(x[1:2:N])
ye = fft(x[2:2:N]).*linphase(0.5, N÷2)

# add them to get the full fft
round.([yo ye yo + ye; yo -ye yo - ye;]; digits=2)

# Real and Imaginary Valued FFTs

In [ ]:
# calculate FFT of just real values
yre = fft(real(x))
yim = fft(imag(x)im)

# FFT is linear, you can add the results
round.([yre yim yre + yim]; digits=2)

In [ ]:
# works the other way too if you have the FFT
y2 =  conj(flip(y))
yre = (y + y2)/2
yim = (y - y2)/2
round.([yre yim]; digits=2)

# Real Valued FFTs

In [ ]:
x = real(x)
round.([x fft(x)]; digits=2)

In [ ]:
x2 = x[1:2:N] + x[2:2:N]im
p = fft(x2)/2
q = conj(flip(p))
w = -im.*linphase(0.5, N÷2)
yo = (p + q)
ye = (p - q).*w
y3 = [yo + ye; yo[1] - ye[1]]
# y3 = [p + p.*w + q - q.*w; yo[1] - ye[1]]

round.(y3; digits=2)

## Real Valued iFFTs

In [ ]:
y4 = [y3; conj(y3[N÷2:-1:2])]
round.(ifft(y4); digits=2)

In [ ]:
# (y2[1] + y2[5])/2 # yo[1]
# (y2[1] - y2[5])/2 # ye[1]
# y2 - (flip(y2))im

_p = y4
# _q = conj(flip(_p))
# _yo = (_p + _q)
# _ye = (_p - _q).*-im.*linphase(0.5, N)

# _yo + _ye

# (flip∘fft)(p)/2

In [ ]:
[[p;0] [q;0] y3]

# Calculating DCTs via the FFT

## DCT-II

In [ ]:
dct2(x) = real(fft([x; reverse(x)])[1:N].*linphase(0.25, N))
dct2(x)

In [ ]:
x2 = real([x; reverse(x)])
# x2 = [1:1:N; N:-1:1]

x2o = x2[1:2:2N]
# x2e = x2[2:2:2N]
# x2e = reverse(x2o) # x2e is also just the reverse of x2o

y2o = fft(x2o)
# y2e = fft(x2e)
# y2e = flip(y2o).*linphase(-1, N) # y2o can be flipped and phased to calculate y2e

# direct computation via evens/odds and a phase shift
# y2 = (y2o + y2e.*linphase(0.5, N)).*linphase(0.25, N)
# but y2e can be substituted out
# y2 = (y2o + flip(y2o).*linphase(-0.5, N)).*linphase(0.25, N)
# simplified by multiplying out the phases
y2p = y2o.*linphase(0.25, N) + flip(y2o).*conj(linphase(0.25, N))

# round.(y2; digits=2)
round.(y2p - dct2(x); digits=2)

In [ ]:
_y2p = dct2(x)
round.([y2p _y2p]; digits=2)

_y2 = _y2p.*linphase(-0.25, N)
round.([y2 _y2]; digits=2)

_y2o = (_y2 - flip(_y2).*linphase(-0.5, N))/2
# _y2o = (_y2p.*linphase(-0.25, N) - flip(_y2p).*linphase(-0.25, N)*im)/2
_y2o[1] = _y2[1]/2 # special case
round.([y2o _y2o]; digits=2)

_x2o = fft(flip(_y2o))/N
round.([x2o _x2o])

# _x = zeros(N)
# _x[1:2:N] = real(_x2[1:N÷2])
# _x[2:2:N] = real(_x2[N÷2+1:N])
# round.([x _x])

In [ ]:
round.([linphase(-0.25, N)*im linphase(-0.25, N)*im]; digits=3)

## DCT-III

In [ ]:
dct3(x) = real(fft([x; 0; -reverse(x); -x[2:N]; 0; reverse(x[2:N])])[2:2:2N])/2
dct3(dct2(x))/2N - x

## DCT-IV

In [ ]:
dct4(x) = real(shift(fft([x; -reverse(x); -x; reverse(x)]), 0.5)[2:2:2N])

norm4 = sqrt(2)/N
# dct4(dct4(x))*norm4^2 - 2x
round.(dct4(x); digits=2)

In [ ]:
foo = [x; -reverse(x); -x; reverse(x)].*linphase(1, 4N)
bar = fft(foo).*cispi.((1:4N)*-0.125*2/N)
round.(bar; digits=2)

# thoughts: this produces an fft([a -a]) = [b, -b] pattern in the output, need to find an identity here